In [3]:
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.metrics import matthews_corrcoef



from sklearn.model_selection import train_test_split
from sklearn import metrics
from seaborn import heatmap
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

In [4]:
dataset=pd.read_csv('final_dataset.csv',header=None)

X=dataset[3].copy()
X = X.str.split(expand=True)

Y = dataset[4].copy()
Y = Y.astype('category')

In [5]:
print(Y.value_counts())


1    1155
0    1095
Name: 4, dtype: int64


In [6]:

X = X.astype(float)
X.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
763    float64
764    float64
765    float64
766    float64
767    float64
Length: 768, dtype: object

less values that expected

In [7]:
def cross_val(model, X, Y):
    sensitivity_scorer = make_scorer(recall_score)
    specificity_scorer = make_scorer(recall_score, pos_label=0)
    MCC=make_scorer(matthews_corrcoef)


    scoring = {'AUC': 'roc_auc', 'Accuracy': "accuracy", "f1": "f1",
                        "Recall": "recall", "Precision": "precision","MCC":MCC, "Average Precision": "average_precision",
                        "Sensitivity": sensitivity_scorer, "Specificity": specificity_scorer}

    scores=cross_validate(model, X, Y, scoring=scoring)

    mean_scores = {metric: values.mean() for metric, values in scores.items()}

    mean_scores_df = pd.DataFrame(list(mean_scores.items()), columns=['Metric', 'Mean Score'])

    return mean_scores_df

In [8]:
#import all scalers 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MaxAbsScaler



In [7]:
scaler=MinMaxScaler()
X_scaled=scaler.fit_transform(X)


lr=LogisticRegression(max_iter=1000)
lr_scores=cross_val(lr,X_scaled,Y)
lr_scores

,Metric,Mean Score
0,fit_time,0.443960
1,score_time,0.013663
2,test_AUC,0.706754
3,test_Accuracy,0.655556
4,test_f1,0.668612
5,test_Recall,0.677922
6,test_Precision,0.660103
7,test_MCC,0.310588
8,test_Average Precision,0.719594
9,test_Sensitivity,0.677922


## Logistic regression

In [9]:
from sklearn.linear_model import LogisticRegressionCV


lr=LogisticRegressionCV(max_iter=10000)
lr_scores=cross_val(lr,X,Y)
lr_scores

,Metric,Mean Score
0,fit_time,28.783056
1,score_time,0.028528
2,test_AUC,0.723303
3,test_Accuracy,0.670222
4,test_f1,0.675153
5,test_Recall,0.667532
6,test_Precision,0.683322
7,test_MCC,0.340656
8,test_Average Precision,0.724390
9,test_Sensitivity,0.667532


## LightGBM

In [10]:
lg = LGBMClassifier(verbose=-1,number_estimators=1000,learning_rate=0.1)

lg_scores=cross_val(lg,X,Y)
lg_scores

,Metric,Mean Score
0,fit_time,3.063211
1,score_time,0.029622
2,test_AUC,0.714962
3,test_Accuracy,0.668000
4,test_f1,0.669930
5,test_Recall,0.657143
6,test_Precision,0.684301
7,test_MCC,0.337004
8,test_Average Precision,0.724908
9,test_Sensitivity,0.657143


## Random forest

In [11]:
rf = RandomForestClassifier()

rf_scores=cross_val(rf,X,Y)
rf_scores

,Metric,Mean Score
0,fit_time,2.433424
1,score_time,0.050693
2,test_AUC,0.706790
3,test_Accuracy,0.653333
4,test_f1,0.651478
5,test_Recall,0.631169
6,test_Precision,0.673440
7,test_MCC,0.308129
8,test_Average Precision,0.713623
9,test_Sensitivity,0.631169


In [20]:
# import gridsearchcv
from sklearn.model_selection import GridSearchCV

# create a dictionary of parameters

param_grid = {

    'n_estimators': [100, 500, 1000, 1500, 2000],
    'max_depth': [2, 3, 4, 5, 6, 7, 8],
    'min_samples_split': [2, 3, 4, 5, 6, 7, 8],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# create Random Forest model
rf_model = RandomForestClassifier()

# Create gridsearch object with various combinations of parameters
rf_gscv = GridSearchCV(estimator=rf_model,
                       param_grid=param_grid,
                       scoring='accuracy',
                       cv=5,
                       verbose=1,
                       n_jobs=-1)

# Fit the gridsearch
rf_gscv.fit(X, Y)

# Get the best parameters

rf_gscv.best_params_

# get the values of best parameters
best_params = rf_gscv.best_params_



Fitting 5 folds for each of 1 candidates, totalling 5 fits
{'n_estimators': 100}


In [ ]:
rf = RandomForestClassifier()

rf_scores=cross_val(rf,X,Y)
rf_scores

## SVC

In [12]:
svc = SVC()
svc_scores=cross_val(svc,X,Y)
svc_scores

,Metric,Mean Score
0,fit_time,0.851723
1,score_time,0.642086
2,test_AUC,0.718302
3,test_Accuracy,0.651556
4,test_f1,0.638261
5,test_Recall,0.600000
6,test_Precision,0.682523
7,test_MCC,0.307472
8,test_Average Precision,0.723851
9,test_Sensitivity,0.600000


# E1 only

In [40]:
#sydromic = pd.read_csv('sfari_ed_01_16_2024.csv')
#sydromic = sydromic[pd.isna(sydromic['gene-score'])].copy() # df with only syndromic genes with no gene score

sfari=pd.read_csv('sfari_ed_01_16_2024.csv')
sfari=sfari[sfari['gene-score'] == 1] # df with only sfari E1 genes

#sfari= pd.concat([sydromic,sfari],ignore_index=True) # df with both syndromic and sfari E1 genes


In [43]:
E1=pd.read_csv('final_dataset.csv',header=None)

#merge the two datasets by ensembl id

E1_dataset=pd.merge(E1,sfari,left_on=1,right_on='ensembl-id')
E1_dataset=E1_dataset[[0,1,2,3,4]]

test_dataset=pd.read_csv('final_dataset.csv',header=None)
test_dataset=test_dataset[test_dataset[4]==0]

#concatenate the two datasets
test_dataset=pd.concat([test_dataset,E1_dataset],ignore_index=True)

In [44]:
X_E1=test_dataset[3].copy()
X_E1 = X_E1.str.split(expand=True)
X_E1 = X_E1.astype(float)


Y_E1 = test_dataset[4].copy()
Y_E1 = Y_E1.astype('category')



In [45]:
Y_E1.value_counts()

0    1095
1     232
Name: 4, dtype: int64

## Scores with E1 and Syd

In [39]:
from sklearn.linear_model import LogisticRegressionCV

lr=LogisticRegressionCV(max_iter=10000,class_weight='balanced')
lr_scores=cross_val(lr,X_E1,Y_E1)
lr_scores

,Metric,Mean Score
0,fit_time,8.680138
1,score_time,0.024363
2,test_AUC,0.813788
3,test_Accuracy,0.779750
4,test_f1,0.592973
5,test_Recall,0.702657
6,test_Precision,0.513409
7,test_MCC,0.457650
8,test_Average Precision,0.588279
9,test_Sensitivity,0.702657


## Scores with only E1

In [47]:
from sklearn.linear_model import LogisticRegressionCV

lr=LogisticRegressionCV(max_iter=10000)
lr_scores=cross_val(lr,X_E1,Y_E1)
lr_scores

,Metric,Mean Score
0,fit_time,7.756386
1,score_time,0.024322
2,test_AUC,0.845519
3,test_Accuracy,0.860613
4,test_f1,0.507714
5,test_Recall,0.423404
6,test_Precision,0.648567
7,test_MCC,0.447558
8,test_Average Precision,0.573722
9,test_Sensitivity,0.423404
